In [5]:
class Neighbourhood:

    def __init__(self, rent, commutescores, neighbourhood_id):

        self.rent = rent
        self.neighbourhood_id = neighbourhood_id

        # commute scores will be a nested list
        # lists are for drive; bus; bike; walk
        # list elements are convenience; speed; affordability; sustainability

        self.commutescores = commutescores

        # this will later be used for identifying commute utilization

        self.annual_utilization = [0, 0, 0, 0]


    def define_residents(self, population):

        # runs yearly to update populations
        # pass in total population of Austin

        self.resident_list = []

        for agent in population:
            if agent.neighbourhood == self.neighbourhood_id:
                self.resident_list.append(agent)


    def monthly_commuteupdate(self):
        
        self.monthly_utilization = [0, 0, 0, 0]
        
        # convenience updates on a monthly basis due to congestion
        for agent in self.resident_list:
            
            self.monthly_utilization[agent.transit - 1] += 1
            
        # track utilization to inform annual investments
        self.annual_utilization += self.monthly_utilization

        # congestion effects

        # cars: all scores effected
        self.commutescores[0] -= self.monthly_utilization[0] * [0.05, 0.05, 0.03, 0.03]

        # bus: mild impacts on convenience and speed
        self.commutescores[1] -= self.monthly_utilization[1] * [0.02, 0.02, 0, 0]

        # bike: very mild impacts on convenience
        self.commutescores[2] -= self.monthly_utilization[2] * [0.01, 0, 0, 0]

        # walk: very mild impacts on convenience
        self.commutescores[3] -= self.monthly_utilization[3] * [0.01, 0, 0, 0]


    def yearly_investment(self, city_priorities):

        # convenience and speed update on a yearly basis due to investment
        # city_priorities is an array for interest in investing in infrastructure for [cars, busses, bikes, pedestrians]
        # subtract 0.1 to reflect deterioration

        for i in range(4):
            self.commutescores[i][0] = self.commutescores[i][0] + self.annual_utilization[i] * city_priorities[i] / 100 - 0.1
            self.commutescores[i][1] = self.commutescores[i][1] + self.annual_utilization[i] * city_priorities[i] / 100 - 0.1
        
        self.annual_utilization = [0, 0, 0, 0]


    def yearly_housing_update(self):

        # first you will want to run this once in main
        # for neighbourhood in all_neighbourhoods:
        # neighbourhood.housing_supply = len(neighbourhood.resident_list)

        # rent increases if demand > supply
        self.rent = self.rent * (self.resident_list / self.housing_supply)

        # for next year, housing will adjust slightly to accommodate demand
        self.housing_supply += (self.resident_list - self.housing_supply) / 10
